[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA2/blob/main/lab3/lab3_parte1.ipynb)
# Práctica 2: Redes neuronales usando Keras
## Parte 1. Creando nuestro primer modelo de red neuronal

En esta práctica vamos a emplear Keras, la API de TensorFlow para construir y entrenar modelos de aprendizaje profundo. Para comenzar, veremos las facilidades que presenta para crear las distintas capas de una red neuronal y como dichas capas se pueden combinar para crear una red neuronal.

# Pre-requisitos

## Instalar paquetes

Para esta primera parte solo necesitaremos NumPy, TensorFlow 2.0 y TensorFlow-Datasets






In [2]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

# La clase *Layer*

Una de las principales abstracciones  en Keras es la clase *Layer*. *Layer* nos permite implementar las operaciones más comunes de las capas de una red neuronal, como actualizar pesos, calcular la función de pérdida y definir la conectividad entre capas.
Una *Layer* encapsula tanto un estado (la matriz de pesos de la capa, i.e., $\mathbf{W}$, y también el vector *bias* $\mathbf{b}$) como una transformación de las entradas a las salidas (una *call* que permite realizar el paso hacia adelante de la capa, *forward pass*). Vamos a crear una capa densamente conectada, es decir, una capa donde todas las entradas están conectadas con todas las salidas. Además, y tal cual se hizo en el *Laboratorio 2*, empleará la función *sigmoide* como función de transferencia.

La mejor manera de implementar nuestra propia capa es extender la clase tf.keras.Layer e implementar:

1. `__init__`, donde se puede realizar la inicialización independiente de la entrada
1. *build*, donde se proporciona la forma (*shape*) de los tensores de entrada y se puede hacer el resto de la inicialización
1. *call*, donde se hacen los cálculos.

No es necesario llamar a *build* para crear las variables, también se pueden crear en `__init__` . Sin embargo, la ventaja de crearlos en *build* es que permite la creación tardía de variables en función del *shape* de las entradas con las que operará la capa.

## Creando una capa
Vamos a crear nuestra propia capa que va a heredar de la clase *Layer* y, por tanto, nos permitirá usarla posteriormente en nuestro *modelo* de red neuronal. Al inicializar esta capa solo le indicaremos el número de salidas, al construirla le pasaremos la *shape* de la entrada e inicializaremos los parámetros aleatoriamente (los pesos $\mathbf{W}$ y el bias $b$). En la llamada (*call*) haremos los cálculos necesarios, teniendo en cuenta que la operación *mathmul* de TensorFlow nos permite realizar la multiplicación de matrices y que la función de transferencia es *sigmoide* (para ver las funciones de activación https://keras.io/api/layers/activations/, iremos usando algunas de ellas a lo largo del curso).

*Nota:* Partes de este código han sido extraídas de MIT 6.S191 Introduction to Deep Learning.


In [3]:
class OurDenseLayer(tf.keras.layers.Layer):
  def __init__(self, n_output_nodes):
    #El único parámetro que inicializamos es el número de salidas de la capa
    super(OurDenseLayer, self).__init__()
    self.n_output_nodes = n_output_nodes

  def build(self, input_shape):
    d = int(input_shape[-1])
    # Definir e inicializar parámetros: una matriz de pesos W y un bias b
    # La inicialización de parámetros es aleatoria
    self.W = self.add_weight(name="weight", shape=[d, self.n_output_nodes], dtype="float32", trainable = True)
    #TODO: declarar el bias
    self.b = self.add_weight(
        name="bias",
        shape=[self.n_output_nodes],
        dtype="float32",
        initializer="zeros",
        trainable=True
    )

  def call(self, x):
    #Calculo de z usando mathmul
    #TO-DO: definir z
    # z = xW + b
    z = tf.matmul(x, self.W) + self.b
    #Aplicamos la función sigmoide
    #TO-DO: definir y
    y = tf.keras.activations.sigmoid(z)
    return y

## Cargamos el conjunto de datos

Vamos a emplear el mismo conjunto de datos que en el *Laboratorio 2*.

In [4]:
# Cargamos el conjunto de datos
ds = tfds.load('german_credit_numeric', split='train')

tamano_lote = 100

elems = ds.batch(tamano_lote)
lote_entrenamiento = next(iter(elems))

vectores_x = tf.cast(lote_entrenamiento["features"], dtype=tf.float32)
etiquetas = tf.cast(lote_entrenamiento["label"], dtype=tf.float32)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/german_credit_numeric/incomplete.08JFZG_1.0.0/german_credit_numeric-train.…

Dataset german_credit_numeric downloaded and prepared to /root/tensorflow_datasets/german_credit_numeric/1.0.0. Subsequent calls will reuse this data.


# Concatenando capas para formar una red
Vamos a crear una red, que llamaremos *model*, usando las tres capas tal y como se hizo en el *Laboratorio 2*:

1. La capa $C_0$ consta de 5 unidades. Recibe como entrada el vector $\mathbf{x}$ y produce como salida el vector $\mathbf{h_0}$. Tiene una matriz de pesos $\mathbf{W_0}$ y un vector de bias $\mathbf{b_0}$.

1. La capa $C_1$ consta de 3 unidades. Recibe como entrada el vector $\mathbf{h_0}$ y produce como salida el vector $\mathbf{h_1}$. Tiene una matriz de pesos $\mathbf{W_1}$ y un vector de bias $\mathbf{b_1}$.

1. La capa $C_2$ consta de 1 unidad. Recibe como entrada el vector $\mathbf{h_1}$ y produce como salida el vector $\mathbf{y}$. Tiene una matriz de pesos $\mathbf{W_2}$ y un vector de bias $\mathbf{b_2}$


In [5]:
tamano_entrada = 24
h0_size = 5
h1_size = 3

class OurNetwork(tf.keras.layers.Layer):
    def __init__(self):
        super(OurNetwork, self).__init__()
        self.layer0 = OurDenseLayer(h0_size)   # #Creamos la primera capa: Capa C0
        self.layer1 = OurDenseLayer(h1_size)   # Capa C1
        self.layer2 = OurDenseLayer(1)         # Capa C2 (salida)

    def call(self, x):
        #TODO - Aquí se debería llamar a las distintas capas haciendo el forward pass
        h0 = self.layer0(x)   # forward en C0
        h1 = self.layer1(h0)  # forward en C1
        y  = self.layer2(h1)  # forward en C2
        return y


model = OurNetwork()
y = model(vectores_x)  # La primera llamada a 'model' creará los pesos

#Comprobamos el numero de pesos que hay
assert len(model.weights) == 6

#Como estos pesos son "entrenables" se encuentran en `trainable_weights`
assert len(model.trainable_weights) == 6


## Entrenamiento del modelo

Al igual que hicimos en el *Laboratorio 2* vamos a emplear las funciones de TensorFlow para ajustar los parámetros de la red (ahora incluidos en nuestro *model*) de modo que se minimice la función de coste, así indicamos:

 1. La función de pérdida que queremos (entropía cruzada)
 1. El método de optimización a utilizar (descenso de gradiente).



In [6]:
#TODO - Indica la función de perdida y el algoritmo de descenso de gradiente

# Función de pérdida: entropía cruzada binaria
fn_perdida = tf.keras.losses.BinaryCrossentropy()

# Optimizador: descenso de gradiente estocástico (SGD)
optimizador = tf.keras.optimizers.SGD(learning_rate=0.01)
# learning_rate -> tamaño del paso que damos en la dirección del gradiente

Utilizamos el mismo bucle de entrenamiento que en el *Laboratorio 2*, pero ahora no tenemos la función *predice* y las *VARIABLES* que teníamos que ir ajustando tampoco se han declarado, ¿qué deberíamos usar?

In [8]:
@tf.function
def paso_entrenamiento(x, y):
    # Declaración del GradientTape que registrará las operaciones
    with tf.GradientTape() as tape:
        # Forward pass para obtener predicciones
        y_pred = model(x)

        # Cálculo de la pérdida utilizando la función que hemos escogido anteriormente
        perdida = fn_perdida(y, y_pred)

    # Consultar los gradientes es tan sencillo como indicarle dos cosas:
    #    1. la función cuyo gradiente queremos obtener
    #    2. la lista de variables respecto a las cuales queremos calcular el gradiente
    # La función nos devolverá una lista con el gradiente correspondiente a cada variable de la lista

    # Calcular gradientes respecto a las variables entrenables del modelo
    gradientes = tape.gradient(perdida, model.trainable_variables)

    # Realizar la actualización de las variables solo requiere esta llamada. Se le pasa una lista de tuplas (gradiente, variable)
    # Actualizar variables con el optimizador
    optimizador.apply_gradients(zip(gradientes, model.trainable_variables))

    # Para poder mostrar la tasa de acierto, la calculamos a cada paso
    fallos = tf.abs(tf.reshape(y, (tamano_lote, 1)) - y_pred)
    tasa_acierto = tf.reduce_sum(1 - fallos)

    # Devolvemos estos dos valores para poder mostrarlos por pantalla cuando estimemos conveniente
    return (perdida, tasa_acierto)



# PROCESO DE ENTRENAMIENTO
num_epochs = 10000

for epoch in range(num_epochs):
    perdida, tasa_error = paso_entrenamiento(vectores_x, etiquetas)

    if epoch % 100 == 99:
        print("Epoch:", epoch, 'Pérdida:', perdida.numpy(), 'Tasa de acierto:', tasa_error.numpy()/tamano_lote)


Epoch: 99 Pérdida: 0.6957387 Tasa de acierto: 0.49876985
Epoch: 199 Pérdida: 0.63712686 Tasa de acierto: 0.5324122
Epoch: 299 Pérdida: 0.60919404 Tasa de acierto: 0.554576
Epoch: 399 Pérdida: 0.5945168 Tasa de acierto: 0.56963986
Epoch: 499 Pérdida: 0.5862899 Tasa de acierto: 0.5802071
Epoch: 599 Pérdida: 0.58190817 Tasa de acierto: 0.5878375
Epoch: 699 Pérdida: 0.57943135 Tasa de acierto: 0.5933761
Epoch: 799 Pérdida: 0.5779653 Tasa de acierto: 0.5974331
Epoch: 899 Pérdida: 0.5770472 Tasa de acierto: 0.6004386
Epoch: 999 Pérdida: 0.5764297 Tasa de acierto: 0.6026892
Epoch: 1099 Pérdida: 0.57597816 Tasa de acierto: 0.60439134
Epoch: 1199 Pérdida: 0.57561827 Tasa de acierto: 0.6056916
Epoch: 1299 Pérdida: 0.5753083 Tasa de acierto: 0.60669535
Epoch: 1399 Pérdida: 0.5750249 Tasa de acierto: 0.6074789
Epoch: 1499 Pérdida: 0.5747546 Tasa de acierto: 0.6080986
Epoch: 1599 Pérdida: 0.57448965 Tasa de acierto: 0.608596
Epoch: 1699 Pérdida: 0.5742256 Tasa de acierto: 0.6090019
Epoch: 1799 Pérd